<a href="https://colab.research.google.com/github/shv-om/Cryptology/blob/main/DES_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [263]:
import numpy as np

# Conversion to Binary

In [264]:
def conbin(msg):
  newstream = []
  for m in msg:
    b = format(int(m, 16), '04b')
    newstream.append(b.replace('0b', ''))
  
  return ''.join(newstream)

In [265]:
def conhex(binstream):
  hexstream = format(int(binstream, 2), '0x')
  
  return str(hexstream)

In [266]:
# Calculating XOR for two given stream
def calc_xor(stream1, stream2):
  newstream = []
  for i in range(len(stream1)):
    newstream.append(str(int(stream1[i]) ^ int(stream2[i])))
  
  return ''.join(newstream)


# Permutation

In [267]:
def calc_permutation(stream, pmatrix):
  pmshape = pmatrix.shape
  pmsize = pmatrix.size

  newstream = np.empty(pmshape, str)
  
  for i in range(pmshape[0]):
    for j in range(pmshape[1]):
      # print(i, j, '-->', pmatrix[i][j]-1)
      newstream[i][j] = stream[pmatrix[i][j]-1]

  return ''.join(newstream.reshape(pmsize))

In [268]:
# Splitting stream into Left and Right equal bits
def split_left_right(stream):
  left_part = stream[:len(stream)//2]
  right_part = stream[len(stream)//2:]

  return ''.join(left_part), ''.join(right_part)

# Key Generation

In [269]:
# Key circular left shift
def key_left_circular_shift(stream, times):
    for i in range(times):
      stream = stream[1:] + stream[0]
    
    return stream

In [270]:
original_key = '133457799BBCDFF1'
keystream = list(conbin(original_key))
print(keystream)
print(len(keystream))

['0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1']
64


In [271]:
pc1 = np.array([[57, 49, 41, 33, 25, 17, 9],
                [1, 58, 50, 42, 34, 26, 18],
                [10, 2, 59, 51, 43, 35, 27],
                [19, 11, 3, 60, 52, 44, 36],
                [63, 55, 47, 39, 31, 23, 15],
                [7, 62, 54, 46, 38, 30, 22],
                [14, 6, 61, 53, 45, 37, 29],
                [21, 13, 5, 28, 20, 12, 4]])

pc1

array([[57, 49, 41, 33, 25, 17,  9],
       [ 1, 58, 50, 42, 34, 26, 18],
       [10,  2, 59, 51, 43, 35, 27],
       [19, 11,  3, 60, 52, 44, 36],
       [63, 55, 47, 39, 31, 23, 15],
       [ 7, 62, 54, 46, 38, 30, 22],
       [14,  6, 61, 53, 45, 37, 29],
       [21, 13,  5, 28, 20, 12,  4]])

In [272]:
pc2 = np.array([[14, 17, 11, 24, 1, 5],
                [3, 28, 15, 6, 21, 10],
                [23, 19, 12, 4, 26, 8],
                [16, 7, 27, 20, 13, 2],
                [41, 52, 31, 37, 47, 55],
                [30, 40, 51, 45, 33, 48],
                [44, 49, 39, 56, 34, 53],
                [46, 42, 50, 36, 29, 32]])

pc2

array([[14, 17, 11, 24,  1,  5],
       [ 3, 28, 15,  6, 21, 10],
       [23, 19, 12,  4, 26,  8],
       [16,  7, 27, 20, 13,  2],
       [41, 52, 31, 37, 47, 55],
       [30, 40, 51, 45, 33, 48],
       [44, 49, 39, 56, 34, 53],
       [46, 42, 50, 36, 29, 32]])

In [273]:
# Calculating Key Permutation
key_permuted = calc_permutation(keystream, pc1)
key_permuted

'11110000110011001010101011110101010101100110011110001111'

In [274]:
# Splitting keystream into left and right part
KeyLeft, KeyRight = split_left_right(key_permuted)
KeyLeft, KeyRight

('1111000011001100101010101111', '0101010101100110011110001111')

In [275]:
# Creating 16 keys pairs by Left shift
key_pairs = []

tempkeypair = KeyLeft, KeyRight

for i in range(1, 17):
  if i in [1, 2, 9, 16]:
    tempkeypair = key_left_circular_shift(tempkeypair[0], 1), key_left_circular_shift(tempkeypair[1], 1)
  else:
    tempkeypair = key_left_circular_shift(tempkeypair[0], 2), key_left_circular_shift(tempkeypair[1], 2)
  
  key_pairs.append(tempkeypair)

key_pairs


[('1110000110011001010101011111', '1010101011001100111100011110'),
 ('1100001100110010101010111111', '0101010110011001111000111101'),
 ('0000110011001010101011111111', '0101011001100111100011110101'),
 ('0011001100101010101111111100', '0101100110011110001111010101'),
 ('1100110010101010111111110000', '0110011001111000111101010101'),
 ('0011001010101011111111000011', '1001100111100011110101010101'),
 ('1100101010101111111100001100', '0110011110001111010101010110'),
 ('0010101010111111110000110011', '1001111000111101010101011001'),
 ('0101010101111111100001100110', '0011110001111010101010110011'),
 ('0101010111111110000110011001', '1111000111101010101011001100'),
 ('0101011111111000011001100101', '1100011110101010101100110011'),
 ('0101111111100001100110010101', '0001111010101010110011001111'),
 ('0111111110000110011001010101', '0111101010101011001100111100'),
 ('1111111000011001100101010101', '1110101010101100110011110001'),
 ('1111100001100110010101010111', '101010101011001100111100011

In [276]:
# Generating keys by adding the pairs and then permuted again with pc2 matrix
list_of_keys = []

for i in range(16):
  temp_key = key_pairs[i][0] + key_pairs[i][1]
  list_of_keys.append(calc_permutation(temp_key, pc2))

# Message Encoding

In [277]:
original_msg = '0123456789ABCDEF'
msgstream = list(conbin(original_msg))
print(msgstream)
print(len(msgstream))

['0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1']
64


In [278]:
# Initial permutation Matrix

IP = np.array([[58, 50, 42, 34, 26, 18, 10, 2],
              [60, 52, 44, 36, 28, 20, 12, 4],
              [62, 54, 46, 38, 30, 22, 14, 6],
              [64, 56, 48, 40, 32, 24, 16, 8],
              [57, 49, 41, 33, 25, 17, 9, 1],
              [59, 51, 43, 35, 27, 19, 11, 3],
              [61, 53, 45, 37, 29, 21, 13, 5],
              [63, 55, 47, 39, 31, 23, 15, 7]])

IP

array([[58, 50, 42, 34, 26, 18, 10,  2],
       [60, 52, 44, 36, 28, 20, 12,  4],
       [62, 54, 46, 38, 30, 22, 14,  6],
       [64, 56, 48, 40, 32, 24, 16,  8],
       [57, 49, 41, 33, 25, 17,  9,  1],
       [59, 51, 43, 35, 27, 19, 11,  3],
       [61, 53, 45, 37, 29, 21, 13,  5],
       [63, 55, 47, 39, 31, 23, 15,  7]])

In [279]:
# E Matrix for Bit Selection process

E = np.array([[32, 1, 2, 3, 4, 5],
              [4, 5, 6, 7, 8, 9],
              [8, 9, 10, 11, 12, 13],
              [12, 13, 14, 15, 16, 17],
              [16, 17, 18, 19, 20, 21],
              [20, 21, 22, 23, 24, 25],
              [24, 25, 26, 27, 28, 29],
              [28, 29, 30, 31, 32, 1]])

P = np.array([[16, 7, 20, 21],
              [29, 12, 28, 17],
              [1, 15, 23, 26],
              [5, 18, 31, 10],
              [2, 8, 24, 14],
              [32, 27, 3, 9],
              [19, 13, 30, 6],
              [22, 11, 4, 25]])

In [280]:
# Initializing 8 S - Boxes

s1 = np.array([[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
              [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
              [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
              [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]])

s2 = np.array([[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
               [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
               [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
               [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]])

s3 = np.array([[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
               [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
               [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
               [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]])

s4 = np.array([[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
               [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
               [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
               [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]])

s5 = np.array([[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
               [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
               [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
               [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]])

s6 = np.array([[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
               [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
               [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
               [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]])

s7 = np.array([[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
               [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
               [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
               [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]])

s8 = np.array([[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
               [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
               [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
               [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]])

s_boxes = [s1, s2, s3, s4, s5, s6, s7, s8]

In [281]:
FP = np.array([[40, 8, 48, 16, 56, 24, 64, 32],
              [39, 7, 47, 15, 55, 23, 63, 31],
              [38, 6, 46, 14, 54, 22, 62, 30],
              [37, 5, 45, 13, 53, 21, 61, 29],
              [36, 4, 44, 12, 52, 20, 60, 28],
              [35, 3, 43, 11, 51, 19, 59, 27],
              [34, 2, 42, 10, 50, 18, 58, 26],
              [33, 1, 41, 9, 49, 17, 57, 25]])

FP

array([[40,  8, 48, 16, 56, 24, 64, 32],
       [39,  7, 47, 15, 55, 23, 63, 31],
       [38,  6, 46, 14, 54, 22, 62, 30],
       [37,  5, 45, 13, 53, 21, 61, 29],
       [36,  4, 44, 12, 52, 20, 60, 28],
       [35,  3, 43, 11, 51, 19, 59, 27],
       [34,  2, 42, 10, 50, 18, 58, 26],
       [33,  1, 41,  9, 49, 17, 57, 25]])

In [282]:
# Calculating Initial Permutation
initstream = calc_permutation(msgstream, IP)
initstream

'1100110000000000110011001111111111110000101010101111000010101010'

In [283]:
# Splitting Message into Left & Right equal bits
MsgLeft, MsgRight = split_left_right(initstream)
MsgLeft, MsgRight

('11001100000000001100110011111111', '11110000101010101111000010101010')

Left and Right Stream Calculation process

In [295]:
# Ln = Rn-1
# Rn = Ln-1 + f(Rn-1,Kn)

newleft = MsgLeft
newright = MsgRight

print("L0 =", conhex(newleft), ", R0 =", conhex(newright))

print("\nRound\t  Left\t\t Right\t\t   F\n")

for k in range(1, len(list_of_keys)+1):

  # Right Stream

  # Converting 32 bits into 48 bits using E bit-selection matrix
  R_after_E = calc_permutation(newright, E)

  # Calculating f => Kn + E(Rn-1)
  rightstream_list = calc_xor(list_of_keys[k-1], R_after_E)
  # print(rightstream_list)

  # Splitting stream 
  rightstream_list = [rightstream_list[j:j+6] for j in range(0, len(rightstream_list), 6)]
  # print("R", k, rightstream_list)

  newstream = ''

  for i in range(len(s_boxes)):
    rightstream = rightstream_list[i]

    # index used to identify elements in s_boxes
    row_bits = int(rightstream[0] + rightstream[-1], 2)
    col_bits = int(rightstream[1:-1], 2)

    # Converting to binary
    new_bits = format(s_boxes[i][row_bits][col_bits], '04b').replace('0b', '')

    newstream += new_bits

  # Permutation with P matrix
  f = calc_permutation(newstream, P)

  # Storing temporarily the New Right Half
  tempright = calc_xor(newleft, f)

  # Assigning right half to Left Stream
  newleft = newright
  
  # Updaing the New right with the recently created right half
  newright = tempright

  
  print(k, '\t', conhex(newleft), '\t', conhex(newright), '\t->', conhex(f))

L0 = cc00ccff , R0 = f0aaf0aa

Round	  Left		 Right		   F

1 	 f0aaf0aa 	 ef4a6544 	-> 234aa9bb
2 	 ef4a6544 	 cc017709 	-> 3cab87a3
3 	 cc017709 	 a25c0bf4 	-> 4d166eb0
4 	 a25c0bf4 	 77220045 	-> bb23774c
5 	 77220045 	 8a4fa637 	-> 2813adc3
6 	 8a4fa637 	 e967cd69 	-> 9e45cd2c
7 	 e967cd69 	 64aba10 	-> 8c051c27
8 	 64aba10 	 d5694b90 	-> 3c0e86f9
9 	 d5694b90 	 247cc67a 	-> 22367c6a
10 	 247cc67a 	 b7d5d7b2 	-> 62bc9c22
11 	 b7d5d7b2 	 c5783c78 	-> e104fa02
12 	 c5783c78 	 75bd1858 	-> c268cfea
13 	 75bd1858 	 18c3155a 	-> ddbb2922
14 	 18c3155a 	 c28c960d 	-> b7318e55
15 	 c28c960d 	 43423234 	-> 5b81276e
16 	 43423234 	 a4cd995 	-> c8c04f98


In [296]:
# Adding after changing position of both half
lastround = newright + newleft

# Inverse of Initial Permutation (Final Permutation)
cipherstream = calc_permutation(lastround, FP)

cipher = conhex(cipherstream)

In [297]:
print("Original Text:", original_msg)
print("Key:", original_key)
print("Encrypted Text:", cipher)

Original Text: 0123456789ABCDEF
Key: 133457799BBCDFF1
Encrypted Text: 85e813540f0ab405
